In [ ]:
pip install rich boto3 pandas

In [14]:
import boto3
from rich import print
import pandas as pd
from datetime import datetime, timedelta

# Creamos el cliente de Lambda en la región us-east-1 (Virginia)
lambda_client = boto3.client('lambda', region_name='us-east-1')
cloudwatch_client = boto3.client('cloudwatch', region_name='us-east-1')

# Definimos la ventana de tiempo de los últimos 3 meses
fin = datetime.now()
inicio = fin - timedelta(days=90)



In [15]:
# Obtenemos la lista de funciones Lambda
respuesta = lambda_client.list_functions()
# print(respuesta)

In [53]:

# Revisamos cada función para ver si ha tenido invocaciones en los últimos 3 meses
invocaciones = []
datos = []
# Obtenemos todas las funciones Lambda usando paginación
paginator = lambda_client.get_paginator('list_functions')
for pagina in paginator.paginate():
    for funcion in pagina.get('Functions', []):
        nombre_funcion = funcion['FunctionName']
        estadisticas = cloudwatch_client.get_metric_statistics(
            Namespace='AWS/Lambda',
            MetricName='Invocations',
            Dimensions=[{'Name': 'FunctionName', 'Value': nombre_funcion}],
            StartTime=inicio,
            EndTime=fin,
            Period=3600 * 24,
            Statistics=['Sum']
        )
        total_invocaciones = sum(dp['Sum'] for dp in estadisticas.get('Datapoints', []))
        if total_invocaciones > 0:
            datos.append(estadisticas)
        invocaciones.append({
            "FunctionName": nombre_funcion,
            "Invocaciones": total_invocaciones
        })

lambdas_totales_df = pd.DataFrame(invocaciones)
lambdas_data = pd.DataFrame(datos)


In [54]:
# Filtramos las funciones Lambda que no tienen invocaciones
lambdas_sin_uso = lambdas_totales_df[lambdas_totales_df['Invocaciones'] == 0]

print(f"\nFunciones Lambda sin invocaciones en los últimos 3 meses: \t {len(lambdas_sin_uso)} / {len(lambdas_totales_df)}")
print(lambdas_sin_uso)


Funciones Lambda sin invocaciones en los últimos 3 meses:        305 / 623

FunctionName  Invocaciones
1      prod-socket-services-v1-prod-getAllUser_v1           0.0
3          prod-socket-services-prod-testSocketDB           0.0
4                                  3n89q29-2s1zkk           0.0
6                                      testInvoke           0.0
7                     super-prod-updateBranchRate           0.0
..                                            ...           ...
615      prod-socket-services-prod-cancelTransfer           0.0
617                   admin-prod-deleteTrabajador           0.0
618                 super-prod-updatePlanSucursal           0.0
621  prod-socket-services-prod-setOrderFeeCourier           0.0
622                      rider-prod-registerRider           0.0

[305 rows x 2 columns]

In [52]:
# Guardamos los nombres de las funciones con invocaciones en un CSV
lambdas_totales_df.sort_values(by='Invocaciones', ascending=False).to_csv('lambdas_invocadas.csv', index=False)

In [ ]:
# Calculamos la suma total de invocaciones
total_invocaciones = lambdas_totales_df['Invocaciones'].sum()

print(f"\nTotal de invocaciones de todas las funciones Lambda: {total_invocaciones:,.0f}")

# Obtenemos configuración y tiempo de uso de las funciones con invocaciones
tiempo_total = 0
memoria_total = 0
for funcion in lambdas_totales_df[lambdas_totales_df['Invocaciones'] > 0]['FunctionName']:
    # Obtenemos configuración
    config = lambda_client.get_function_configuration(FunctionName=funcion)
    memoria = config['MemorySize']
    timeout = config['Timeout']
    
    # Calculamos tiempo de ejecución total
    metricas_duracion = cloudwatch_client.get_metric_statistics(
        Namespace='AWS/Lambda',
        MetricName='Duration',
        Dimensions=[{'Name': 'FunctionName', 'Value': funcion}],
        StartTime=inicio,
        EndTime=fin,
        Period=3600*24,
        Statistics=['Average', 'Sum']
    )
    
    if metricas_duracion['Datapoints']:
        tiempo_total += sum(dp['Sum'] for dp in metricas_duracion['Datapoints'])
        memoria_total += memoria
# Guardamos los resultados en un CSV
resultados = pd.DataFrame({
    'Metrica': ['Tiempo total de ejecución (segundos)', 'Memoria total configurada (MB)'],
    'Valor': [tiempo_total/1000, memoria_total]
})
resultados.to_csv('metricas_lambda.csv', index=False)


In [47]:
lambdas_sin_uso["FunctionName"].sort_values().to_csv('lambdas_sin_uso.csv', index=False, header=False)